In [1]:
# https://pypi.org/project/torch-geometric/
# pip install torch
# pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
# pip install torch-geometric


# https://towardsdatascience.com/louvains-algorithm-for-community-detection-in-python-95ff7f675306
import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import Counter 

In [2]:
# plot the data
from utils.visual import plot

# plot(0) 

In [3]:
import re

def find_train_ego_index():
    pre, pree = [], []
    for item in train.files:
        pre.append(int(re.sub('\D', '',item )))

    for i in egonet.files:
        pree.append(int(re.sub('\D', '',i )))
        
    train_ego_index = []
    for i in range(len(pree)):
        if pree[i] in pre:
            train_ego_index.append(i)
    return train_ego_index, pree, pre

In [4]:
# self-made packages
from utils.data import data_preprocess


train = data_preprocess("\Training")
train_list, train_dict = train.train_data()
df = pd.DataFrame(data=train_dict)

In [5]:
egonet = data_preprocess("\egonets")
ego_list, user_list = egonet.ego_data()

In [6]:
# 這個是指應該將哪些 ego_list 用在 training，方便輸入0,1,2，就不用輸入 239、345、661...
train_ego_index,ego_user, train_user  = find_train_ego_index()

## Data Preparation

In [7]:
from utils.user import users

users = users()
index = user_list[train_ego_index[2]] # 更改數字可以挑選要哪一個 training data 

# edge information EX: (240, 241), (240, 242)...
edge1 = egonet.edge_data(train_ego_index[0])

In [8]:
# users = pd.read_csv('df_new_friend.csv') #把註解拿掉代表讀取新的feature

In [9]:
users = users.drop(columns=[
                            'education;classes;from;name',
                            'education;classes;name',
                            'education;classes;with;name',
                            'education;concentration;name',
                            'education;degree;name',
                            'education;school;name',
                            'education;with;name',
                            'education;year;name',
                            'hometown;name',
                            'languages;name',
                            'location;name',
                            'work;employer;name',
                            'work;from;name',
                            'work;location;name',
                            'work;position;name',
                            'work;projects;from;name',
                            'work;projects;name',
                            'work;projects;with;name',
                            'work;with;name'
                            ])


In [10]:
users_friend = pd.read_csv('df_new_friend.csv')

In [11]:
users_friend = users_friend.drop(columns=[
                            'education;classes;from;name;friend',
                            'education;classes;name;friend',
                            'education;classes;with;name;friend',
                            'education;concentration;name;friend',
                            'education;degree;name;friend',
                            'education;school;name;friend',
                            'education;with;name;friend',
                            'education;year;name;friend',
                            'hometown;name;friend',
                            'languages;name;friend',
                            'location;name;friend',
                            'work;employer;name;friend',
                            'work;from;name;friend',
                            'work;location;name;friend',
                            'work;position;name;friend',
                            'work;projects;from;name;friend',
                            'work;projects;name;friend',
                            'work;projects;with;name;friend',
                            'work;with;name;friend'
                            ])


In [12]:
users_target = pd.read_csv('df_new_target.csv')

In [13]:
users_target = users_target.drop(columns=[
                            'education;classes;from;name;target',
                            'education;classes;name;target',
                            'education;classes;with;name;target',
                            'education;concentration;name;target',
                            'education;degree;name;target',
                            'education;school;name;target',
                            'education;with;name;target',
                            'education;year;name;target',
                            'hometown;name;target',
                            'languages;name;target',
                            'location;name;target',
                            'work;employer;name;target',
                            'work;from;name;target',
                            'work;location;name;target',
                            'work;position;name;target',
                            'work;projects;from;name;target',
                            'work;projects;name;target',
                            'work;projects;with;name;target',
                            'work;with;name;target'
                            ])

In [14]:
users = pd.concat([users,users_friend],axis = 1)

In [15]:
users = pd.concat([users,users_target],axis = 1)

## Predict Data

In [16]:
circlesP = {}

# 這裡是整理所有的 predict data，但因為目前沒有所以暫時用 training 的代替
for i in range(len(train_dict)):
    uidP,friendsP = train_dict[i]['UserId'], train_dict[i]['Predicted'] # predict 
    circlesP[int(uidP)] = [set([int(x) for x in c.split()]) for c in friendsP.split(';')]

## Connected Component
#### 測試connected component

In [17]:
from connected_components import findCommunities
import sys, os

def connected_componet(target_dir):
    egoUser = -1
    try:
        egoUser = int(re.sub("\D", '', target_dir))
    except Exception as e:
        print("Expected files to be names 'X.egonet' where X is a user ID")

    cs = list(findCommunities(target_dir))

    if len(cs) == 0:
        cs = [set(adj.keys())]
    cs = [' '.join([str(y) for y in x]) for x in cs]
    res = str(egoUser) + ',' + ';'.join(cs)
    
    return res
    

In [18]:
def process(test_ego_num):
    return [{
            'UserId': test_ego_num.split(',')[0],
            'Predicted': test_ego_num.split(',')[1]
            }]


In [19]:
connect_egonet = []
for i in range(len(train.files)):
    target_dir = os.getcwd()+"\\egonets\\"+train.files[i].replace('circles', 'egonet')
    test_ego_num = connected_componet(target_dir)
    test = process(test_ego_num)
    connect_egonet+=(test)

# Caculate Loss
### 之後可能要放進一個mode裡面當作 objective function 之類的，這邊先暫時表達一次性的評分

## Model

In [20]:
from sklearn.cluster import KMeans

def exe(n_clusters):
    km = KMeans(
        n_clusters=n_clusters, init='random',
        n_init=10, max_iter=300, 
        tol=1e-04, random_state=2
    )
    final, indexx = [], []
    for i in range(len(train_ego_index)):
        index = user_list[train_ego_index[i]] # 更改數字可以挑選要哪一個 training data 
        test = users.iloc[index]

        X = np.array(test).astype(np.int64)
        return X
        y_km = km.fit(X)
        predict = y_km.predict(X)
        final.append(predict)
        indexx.append(index)
    return final, indexx

In [21]:
from socialCircles_metric import loss1

def temperal(train_dict):
        
    circlesG = {}

    for i in range(len(train_dict)):
        uidG,friendsG = train_dict[i]['UserId'], train_dict[i]['Predicted'] # ground truth
        circlesG[int(uidG)] = [set([int(x) for x in c.split()]) for c in friendsG.split(';')] 
        
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                print("Ground-truth has prediction for circle", k, "but prediction does not")
                break

            l = loss1(circlesG[k], circlesP[k])
            #print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)

In [22]:
def data(n_clusters):

    final, index = exe(n_clusters)
    
    resss = []
    for k in range(len(final)):
        fina = ""
        for i in range(n_clusters):
            tem = ''

            for j in range(len(final[k])):
                if final[k][j] == i:
                    tem+= ' ' + str(index[k][j])
            fina += tem.strip() + ';'
        resss.append(fina)
        
    transform_label = []
    
    for i in range(len(index)):
        transform_label.append({"UserId":index[i][0]-1, "Predicted":resss[i].strip()[:-1]})
        
    return transform_label

In [23]:
temperal(connect_egonet)

total loss for all users = 17606


In [24]:
temperal(train_dict)

total loss for all users = 0


In [25]:
index = train_ego_index[30]
first_user = user_list[index][0]

In [26]:
from collections import Counter

def count_similarity(a, b):
    count = 0
    for i in range(len(a)):
        if (a[i] != -1) and (a[i] == b[i]):
            count+=1
            
    return count/(len(a) - Counter(a)[-1]) # 分母為去除掉 -1 的個數

In [27]:
test = users.iloc[user_list[index]]

In [28]:
res = []
for j in range(len(user_list[index])):
    user, friend = ego_list[index][j][0], ego_list[index][j][1:]
    temp1 = []
    for i in range(len(friend)):
        temp1.append(count_similarity(test.iloc[user-first_user], test.iloc[friend[i] - first_user]))
    res.append(temp1)

In [30]:
def process_res(res, first_user):
    temp = ''
    index = 0
    a = ''
    for i in range(len(res)):
        if index != res[i][1]:
            index = res[i][1]
            temp += a.strip() + ';'
            a = ''
        a += ' '+str(res[i][0])
    temp += a.strip()
        
    return [{'UserId':first_user-1, 'Predicted':temp}]

In [31]:
from socialCircles_metric import loss1

def another_temperal(circlesG):
        
    totalLoss = 0
    for k in circlesP.keys():
        try:
            if k not in circlesG:
                print("Ground-truth has prediction for circle", k, "but prediction does not")
                #break

            l = loss1(circlesG[k], circlesP[k])
            #print("loss for user", k, "=", l)
            totalLoss += l
        except:
            continue
            
    print("total loss for all users =", totalLoss)
    return totalLoss

In [32]:
import community
from utils.visual import plot, draw, colors
import networkx as nx
import networkx.algorithms.community as nx_comm
# from sklearn.metrics import pairwise_distances
from itertools import combinations

# Create a graph using NetworkX
def do(W):
    aa=[]
    bb={}
    
    for i in range(len(train_ego_index)):
        G = nx.Graph()
        index = train_ego_index[i]
        first_user = user_list[index][0]
        temp_path = os.path.join(egonet.target_dir, egonet.files[index]) #51
        G = draw(temp_path)
        # G.add_nodes_from(user_list[train_ego_index[i]])
        for u,v in combinations(G.nodes(),2):
            if (u,v) in G.edges():
                SIM = np.array((users.iloc[u].to_list()) == np.array(users.iloc[v].to_list())) & (np.array(users.iloc[u].to_list()) != '-1')
                q = np.sum((np.array(users.iloc[u].to_list()) != '-1') & (np.array(users.iloc[v].to_list()) != '-1'))
                similarity = np.sum(SIM * W)
                G.add_edge(u,v,weight=similarity/q)

        bb[first_user-1] = nx_comm.louvain_communities(G,threshold=0.1 , seed=123)

#         partition = community.best_partition(G)
#         res = sorted(partition.items(), key=lambda item: item[1])
#         aa+=process_res(res, index)
    return bb #aa

In [33]:
W = np.ones(len(users.columns))

In [34]:
aaa = do(W)

In [35]:
another_temperal(aaa)

total loss for all users = 12942


12942

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
import pygad

def fitness_func(solution, solution_idx):
        return 99999-another_temperal(do(solution))


ga_instance = pygad.GA(num_generations=20,
                        num_parents_mating=20, 
                        fitness_func=fitness_func,
                        sol_per_pop=50, 
                        num_genes=len(users.columns),
                        init_range_low=0,
                        init_range_high=1,
                        # mutation_percent_genes=1,
                        # keep_parents=5
                        )

ga_instance.run()

solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Fitness: {solution_fitness}".format(solution_fitness=solution_fitness))
print(another_temperal(do(solution)))

ga_instance.plot_result()
